## Day 9: Smoke Basin

These caves seem to be lava tubes. Parts are even still volcanically active; small hydrothermal vents release smoke into the caves that slowly settles like rain.

If you can model how the smoke flows through the caves, you might be able to avoid it and be that much safer. The submarine generates a heightmap of the floor of the nearby caves for you (your puzzle input).

Smoke flows to the lowest point of the area it's in. For example, consider the following heightmap:

```
2199943210
3987894921
9856789892
8767896789
9899965678
```

Each number corresponds to the height of a particular location, where 9 is the highest and 0 is the lowest a location can be.

Your first goal is to find the low points - the locations that are lower than any of its adjacent locations. Most locations have four adjacent locations (up, down, left, and right); locations on the edge or corner of the map have three or two adjacent locations, respectively. (Diagonal locations do not count as adjacent.)

In the above example, there are four low points, all highlighted: two are in the first row (a 1 and a 0), one is in the third row (a 5), and one is in the bottom row (also a 5). All other locations on the heightmap have some lower adjacent location, and so are not low points.

The risk level of a low point is 1 plus its height. In the above example, the risk levels of the low points are 2, 1, 6, and 6. The sum of the risk levels of all low points in the heightmap is therefore 15.

Find all of the low points on your heightmap. What is the sum of the risk levels of all low points on your heightmap?



In [2]:
import numpy as np

def read_heightmap(filename):
    """
    Read heightmap file into numpy matrix
    
    
    """
    
    with open(filename, "r") as infile:
        height_raw = [list(map(int, l.strip())) for l in infile]
        
    # Pad with 10s around the edges to correctly treat points on the edges - 
    # points outside the map always considered higher
    height_raw = [[10]*len(height_raw[0])] + height_raw
    height_raw += [height_raw[0]]
    height_raw = [[10] + x + [10] for x in height_raw]
    
    # Convert to numpy array for ease of calculations
    height_matrix = np.matrix(height_raw)
        
    return height_matrix

def find_lowest_points(filename):
    """
    Find lowest points in heightmap

    
    """
        
    # Load height map into numpy matrix
    height_matrix = read_heightmap(filename)
        
    # Number of rows and columns of map (pre-padding)
    map_r = height_matrix.shape[0]-1
    map_c = height_matrix.shape[1]-1
    
    # Initialise matrix as True for each point on the map
    matrix_lower = np.full((map_r-1, map_c-1), True)
    
    # Compare each point to its neighbour by shifting matrix across in each direction
    # Updating the truth matrix each time. The only points remaining as True are
    # the lowest points we are looking for
    for shift_dir in [-1, 1]:
        for shift_axis in [0, 1]:
            check_neighbours = height_matrix[1:map_r, 1:map_c] < np.roll(height_matrix, shift_dir, axis=shift_axis)[1:map_r, 1:map_c]
            matrix_lower = np.logical_and(matrix_lower, check_neighbours)
    
    return sum([x+1 for x in height_matrix[1:map_r, 1:map_c][matrix_lower].tolist()[0]])

    
find_lowest_points("day9-input.txt")


491

## Part Two 

Next, you need to find the largest basins so you know what areas are most important to avoid.

A basin is all locations that eventually flow downward to a single low point. Therefore, every low point has a basin, although some basins are very small. Locations of height 9 do not count as being in any basin, and all other locations will always be part of exactly one basin.

The size of a basin is the number of locations within the basin, including the low point. The example above has four basins.

The top-left basin, size 3:

```
2199943210
3987894921
9856789892
8767896789
9899965678
```

The top-right basin, size 9:

```    
2199943210
3987894921
9856789892
8767896789
9899965678
```

The middle basin, size 14:

```
2199943210
3987894921
9856789892
8767896789
9899965678
```

The bottom-right basin, size 9:

```
2199943210
3987894921
9856789892
8767896789
9899965678
```

Find the three largest basins and multiply their sizes together. In the above example, this is 9 * 14 * 9 = 1134.

What do you get if you multiply together the sizes of the three largest basins?

In [77]:
def neighbours(points):
    """
    Returns neighbours for a point
    
    """
    
    neighbs = set()
    
    for point in points:
        neighbs = neighbs.union({(point[0]+1, point[1]),
                   (point[0]-1, point[1]), 
                   (point[0], point[1]+1),
                   (point[0], point[1]-1)})
    
    return neighbs


def count_basins(filename):
    """
    Find three largest basins
    
    """
    
    # Load height map into numpy matrix
    height_matrix = read_heightmap(filename)
        
    # Number of rows and columns of map (pre-padding)
    map_r = height_matrix.shape[0]-1
    map_c = height_matrix.shape[1]-1
    
    # Initialise matrix as True for each point on the map
    matrix_lower = np.full((map_r-1, map_c-1), True)
    
    # Compare each point to its neighbour by shifting matrix across in each direction
    # Updating the truth matrix each time. The only points remaining as True are
    # the lowest points we are looking for
    for shift_dir in [-1, 1]:
        for shift_axis in [0, 1]:
            check_neighbours = height_matrix[1:map_r, 1:map_c] < np.roll(height_matrix, shift_dir, axis=shift_axis)[1:map_r, 1:map_c]
            matrix_lower = np.logical_and(matrix_lower, check_neighbours)
    
    # Indices of lowest points
    lowest_ind = np.where(matrix_lower)
    
    # Store basins
    basin_sizes = []
    
    # Check basin for each of the lowest points
    for i in list(zip(lowest_ind[0],lowest_ind[1])):
        
        # Remember to add 1 to each coordinate as we're working in the 
        # adapted height map with 10's on the borders
        
        # Now to find the basin this point is in...
        
        # Initialise basin with current points
        basin = {(i[0]+1, i[1]+1)}
        
        new_points = neighbours(basin)
                
        i = 0
        
        # While there are neighbours < 9...
        while any([height_matrix[x] < 9 for x in new_points]):
            
            # Add points < 9 to the basin
            good_points = {x for x in new_points if height_matrix[x] < 9}
            
            basin = basin.union(good_points)

            # Find neighbours of newly added basin points
            new_points = neighbours(good_points)
            
            new_points = new_points.difference(basin)
            
        basin_sizes += [len(basin)]

    basin_sizes.sort(reverse = True)
    
    
    return basin_sizes[0]*basin_sizes[1]*basin_sizes[2]

    
count_basins("day9-example.txt")



1134

In [78]:
count_basins("day9-input.txt")



1075536